### Colab Setup

In [3]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

if IN_COLAB:
  # Mount the Google Drive at mount
  mount='/content/drive'
  print("Colab: mounting Google drive on ", mount)

  drive.mount(mount)

  # Switch to the directory on the Google Drive that you want to use
  import os
  drive_root = mount + "/My Drive/Colab Notebooks/Project23"
  
  # Create drive_root if it doesn't exist
  create_drive_root = True
  if create_drive_root:
    print("\nColab: making sure ", drive_root, " exists.")
    os.makedirs(drive_root, exist_ok=True)
  
  # Change to the directory
  print("\nColab: Changing directory to ", drive_root)
  %cd $drive_root
  !pwd
  !pip install -r requirements.txt

  from IPython.display import JSON
  from google.colab import output
  from subprocess import getoutput
  import os
  
  #@title jQuery Terminal's [Features](https://terminal.jcubic.pl/)

  def shell(command):
    if command.startswith('cd'):
      path = command.strip().split(maxsplit=1)[1]
      os.chdir(path)
      return JSON([''])
    return JSON([getoutput(command)])
  output.register_callback('shell', shell)

## Import Tensorflow Libraries

In [4]:
# %%
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gc
import memory_profiler
import numpy
# import cupy as np
import matplotlib.pyplot as plt
from matplotlib import style
#style.use('classic')
import PIL
import scipy
import more_itertools
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import cv2 
import glob
import os
import time
from numpy import genfromtxt, asarray, savez_compressed, load
from sklearn.model_selection import train_test_split
from PIL import Image 
from io import StringIO


Inapkdgma


In [5]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model

try:
  IN_COLAB = True
  from google.colab import drive
  from tensorflow.keras.optimizers.legacy import RMSprop,Adam,SGD,Adadelta,Adagrad,Adamax
except:
  IN_COLAB = False
  from tensorflow.keras.optimizers import RMSprop,Adam,SGD,Adadelta,Adagrad,Adamax

from tensorflow.keras.optimizers.legacy import RMSprop,Adam,SGD,Adadelta,Adagrad,Adamax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.compat.v1.keras.backend import set_session
from tensorflow.keras.utils import plot_model, to_categorical, normalize
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

### Check for available GPUs and configure Tensorflow session to utilize GPU

In [6]:
try:
  # tf.debugging.experimental.enable_dump_debug_info('.', tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)
  # tf.debugging.set_log_device_placement(True)
  from tensorflow.python.client import device_lib

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

  config = tf.compat.v1.ConfigProto()
  config.gpu_options.allow_growth = True
  config.gpu_options.per_process_gpu_memory_fraction = 0.1
  sess = tf.compat.v1.InteractiveSession(config=config)
  set_session(sess)
  print(device_lib.list_local_devices())
  gpus = tf.config.experimental.list_physical_devices('GPU')
  for gpu in gpus:
    try:
    
      tf.config.experimental.set_memory_growth(gpu, True)
      # Restrict TensorFlow to only use the first GPU
      tf.config.set_visible_devices(gpus[0], 'GPU')
      logical_gpus = tf.config.list_logical_devices('GPU')
      print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
      # Visible devices must be set before GPUs have been initialized
      print(e)

except Exception as error:
    print("Error trying to configure computing device.")
    print(error)

Found GPU at: /device:GPU:0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16923097111553654001
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4841275392
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7069230853908044233
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]
1 Physical GPUs, 1 Logical GPU


In [7]:
if IN_COLAB:
  DATASET_FOLDER = './data/dataset1/'
  DATA_FOLDER = './data/'
else:
  DATASET_FOLDER = './../data/dataset1/'
  DATA_FOLDER = './../data/'

INPUT_DATA = 'Image_Input'
OUTPUT_DATA = 'Image_Output'


In [8]:
input_image_path = os.path.join(DATASET_FOLDER, INPUT_DATA)
input_filepaths=glob.glob(os.path.join(input_image_path, f'*.jpg'))
len(input_filepaths)

23

In [9]:
output_image_path = os.path.join(DATASET_FOLDER, OUTPUT_DATA)
output_filepaths=glob.glob(os.path.join(input_image_path, f'*.jpg'))
len(output_filepaths)


23

In [10]:
import cupy as cp

In [13]:
input_im_data = []
SIZE=0
for image_file in input_filepaths:
    # image = Image.open(image_file)
    image = cv2.imread (image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.reshape(image.shape[0],image.shape[1],image.shape[2])
    size = image.size
    SIZE = size if size>SIZE else SIZE
    print(f'image size: {image.size}')
    # plt.imshow(image)
    # plt.show()
    # if image.shape != (173, 267, 3):
    #     print(myFile)
    #     plt.imshow(image)
    #     plt.show()
    #     print(image.shape)
    #     #os.remove(myFile)
    #     prepData()
    #     break
    input_im_data.append(cp.asnumpy(image))
#print(count)


    

image size: 1645056
image size: 1762560
image size: 1628928
image size: 2133504
image size: 1767168
image size: 1760256
image size: 1960704
image size: 2221056
image size: 1767168
image size: 3213000
image size: 1760256
image size: 1769472
image size: 1815552
image size: 2012160
image size: 2064384
image size: 2045952
image size: 1900800
image size: 1610496
image size: 1631232
image size: 1976400
image size: 2138400
image size: 2419200
image size: 1956864


In [14]:
SIZE

3213000

In [ ]:
# raise Exception(f'Wrong file count: Expected 3 files but got {flen} {file}. Please rectify')
           

            
           for myFile in files:
                #print(myFile)
                image=Image.open(myFile)
                image = cv2.imread (myFile)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = image.reshape(image.shape[0],image.shape[1],image.shape[2])
                #plt.imshow(image)
                #plt.show()
                if image.shape != (173, 267, 3):
                    print(myFile)
                    plt.imshow(image)
                    plt.show()
                    print(image.shape)
                    #os.remove(myFile)
                    prepData()
                    break
                im_data.append(np.asnumpy(image))
                proceed = True
            #print(count)
    